In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from gprm import ReconstructionModel
import pygmt
import geopandas as gpd
import sys
sys.path.append('/Users/simon/GIT/degenerative_art/')
import map_effects as me
sys.path.append('/Users/simon/GIT/agegrid-0.1/')
from run_paleo_age_grids import run_paleo_age_grids
import xarray as xr
from xrspatial import proximity
import gprm.utils.paleogeography as pg
import pygplates


from octopus import generate_seafloor_bathymetry, generate_land_topography, merge_topography_and_bathymetry
from octopus import add_seamount_trails, generate_seafloor_fabric
from octopus import generate_slab_earthquakes, generate_magnetic_map
from octopus import smooth_seafloor
from octopus import make_noise_grid


%load_ext autoreload
%autoreload 2


In [ ]:
Atlantis = ReconstructionModel()

Atlantis.add_continent_polygons('/Users/simon/Documents/2022IMAS-OUC_SOMG/PracFiles/Atlantis2/Atlantis2_continents.gpml')
Atlantis.add_dynamic_polygons('/Users/simon/Documents/2022IMAS-OUC_SOMG/PracFiles/Atlantis2/Atlantis2_topologies.gpml')
Atlantis.add_dynamic_polygons('/Users/simon/Documents/2022IMAS-OUC_SOMG/PracFiles/Atlantis2/Atlantis2_geometries.gpml')
Atlantis.add_rotation_model('/Users/simon/Documents/2022IMAS-OUC_SOMG/PracFiles/Atlantis2/Atlantis2_rotations_rel.rot')

seafloor_age_file = './Atlantis2/masked/Atlantis2_seafloor_age_mask_0.0Ma.nc'
final_grd_sampling = 0.25


In [ ]:
### 1 run paleoagegrid

#run_paleo_age_grids('/Users/simon/GIT/agegrid-0.1/config_files/config_Atlantis.yaml')

seafloor_age = pygmt.grdsample(seafloor_age_file,
                               region='d', spacing=final_grd_sampling)

seafloor_depth = generate_seafloor_bathymetry(seafloor_age, 
                                              final_grd_sampling)

seafloor_depth.plot()


In [ ]:
### 2 run make mountain ranges

reconstruction_model = Atlantis
reconstruction_time = 0.
min_distance_to_coastlines=0
max_distance_to_trenches=1200000
sampling = final_grd_sampling

#orogeny_core.plot()


(land, 
 mountain_core, 
 orogeny_core, 
 topography) = generate_land_topography(
    reconstruction_model, reconstruction_time,
    final_grd_sampling, min_distance_to_coastlines=0, max_distance_to_trenches=1200000,
    orogeny_geometries='/Users/simon/Documents/2022IMAS-OUC_SOMG/PracFiles/Atlantis2/Atlantis2_OrogenicBelts.gpml')

#land.plot()
#mountain_core.plot()
topography.plot()


In [ ]:
from gprm.utils.proximity import polyline_proximity

prox_sz = polyline_proximity(Atlantis.plate_snapshot(0.).get_boundary_features(boundary_types=['subduction']),
                             spacing=final_grd_sampling)


topography_noise = make_noise_grid(lmin=10, lmax=50, exponent=-1.5, scaling=1., spacing='{:f}d'.format(final_grd_sampling))
topography.data = topography.data * np.abs(topography_noise)

merge = merge_topography_and_bathymetry(seafloor_depth, topography, 
                                        prox_sz)

merge.plot(cmap='viridis', vmin=-6000, vmax=4000.)



In [ ]:
seafloor_fabric = generate_seafloor_fabric(seafloor_age, final_grd_sampling, target_range=1000)

#filt_grid = filt_grid+make_noise_grid(scaling=2.)

#fig,ax = plt.subplots(figsize=(20,10))
#filt_grid.plot(vmin=-2,vmax=2, ax=ax)

seafloor_fabric = seafloor_fabric.where(np.isfinite(seafloor_fabric), 0)

#seafloor_fabric.plot()
#print(seafloor_fabric.min(), seafloor_fabric.max())


seafloor_fabric = smooth_seafloor(seafloor_age, seafloor_fabric, 
                                  scaling=0.08, vary_kernel_size=True, target_range=10000)

seafloor_fabric.plot(vmin=-1000, vmax=1000)
plt.show()
#'''
mergef = merge + seafloor_fabric
mergef.plot(vmin=-6000, vmax=4000)
plt.show()

plt.plot(mergef.data[290,:])
#'''
#plt.plot(seafloor_fabric.data[180,:])


In [ ]:
hot_spot_points = {
    'A':(10,-10), 
    'B':(-20,20), 
    'C':(10,40)
}

(merge_f_smt,
 hot_spot_trails) = add_seamount_trails(Atlantis, 
                                        mergef, 
                                        hot_spot_points, 
                                        final_grd_sampling, 
                                        filter_length_km=700)

merge_f_smt.plot(cmap='viridis', vmin=-8000, vmax=4000)


In [ ]:
### Synthetic drill holes?


### Present-day GPS velocity


In [ ]:
tmi_grid_sampling = 0.5
tmi = generate_magnetic_map(Atlantis, 
                            pygmt.grdsample(seafloor_age_file,
                                            region='d', spacing=tmi_grid_sampling), 
                            tmi_grid_sampling)


fig,ax = plt.subplots(figsize=(12,6))
tmi.plot(cmap='seismic', ax=ax, vmin=-200, vmax=200)
plt.show()


In [ ]:
slab_earthquakes = generate_slab_earthquakes(Atlantis)

fig = plt.figure(figsize=(12,6))
ax = fig.add_subplot(111)
cm = ax.scatter(slab_earthquakes.geometry.x, 
                slab_earthquakes.geometry.y, 
                c=slab_earthquakes.depth, 
                s=50,
                cmap='magma_r', vmin=0., vmax=450)
ax.set_aspect('equal')
fig.colorbar(cm)
fig.show()



In [ ]:
merge_f_smt.to_netcdf('./Atlantis2_Database/topography_bathymetry.nc')
tmi.to_netcdf('./Atlantis2_Database/total_magnetic_intensity.nc')

slab_earthquakes.to_file('./Atlantis2_Database/Earthquakes.geojson')



In [ ]:
from octopus import generate_plate_boundary_earthquakes

plate_boundary_earthquakes = generate_plate_boundary_earthquakes(Atlantis, int(2e4))

plate_boundary_earthquakes.to_file('./Atlantis2_Database/PlateBoundary_Earthquakes.geojson')



In [ ]:

ns = make_noise_grid(lmin=10, lmax=80, exponent=-1.5, scaling=1., spacing='0.25d')

ns2 = make_noise_grid(lmin=3, lmax=30, exponent=-3, scaling=0.5, spacing='0.25d')

#tmp = topography + ns
#tmp.plot()
#ns.plot()

ns = ns+1
ns2 = ns2+1

tmp = topography.copy()
tmp.data = tmp.data * np.abs(ns.data)
tmp.data = tmp.data * np.abs(ns2.data)

tmp.plot(vmax=6000)


In [ ]:
#ns2 = make_noise_grid(lmin=4, lmax=30, exponent=-3, scaling=0.4, spacing='0.25d')
ns2.plot()


In [ ]:
from gprm.utils.create_gpml import gpml2gdf
from octopus import unreconstruct_geodataframe, generate_climate_sensitive_deposits, generate_weighted_random_points

continent_polygons = gpml2gdf(reconstruction_model.continent_polygons[0])

fig,ax = plt.subplots()

evaporites = []
glacial_deposits = []
arc_volcanism = []

for reconstruction_time in np.arange(200,-1,-1):
    
    print(reconstruction_time)
    #mountain_core, land = get_paleogeography_rasters(reconstruction_model, reconstruction_time)
    (land, 
     mountain_core, 
     orogeny_core, 
     topography) = generate_land_topography(
        reconstruction_model, reconstruction_time,
        final_grd_sampling, min_distance_to_coastlines=0, max_distance_to_trenches=1200000,
        orogeny_geometries='/Users/simon/Documents/2022IMAS-OUC_SOMG/PracFiles/Atlantis2/Atlantis2_OrogenicBelts.gpml')

    reconstructed_polygons = reconstruction_model.reconstruct(continent_polygons, 
                                                              reconstruction_time)

    # Evaporites
    gdf = generate_climate_sensitive_deposits(land, central_lat=20, lat_std=5, n_points=50)
    ur_gdf = unreconstruct_geodataframe(gdf, reconstruction_model, reconstructed_polygons, reconstruction_time)
    ur_gdf.plot(ax=ax, color='orange', markersize=2)
    evaporites.append(ur_gdf)
    
    # Glacial Deposits
    gdf = generate_climate_sensitive_deposits(land, central_lat=90, lat_std=15, n_points=100)
    ur_gdf = unreconstruct_geodataframe(gdf, reconstruction_model, reconstructed_polygons, reconstruction_time)
    if ur_gdf is not None:
        ur_gdf.plot(ax=ax, color='blue', markersize=5)
        glacial_deposits.append(ur_gdf)
    
    # Volcanos
    volcano_points = generate_weighted_random_points(mountain_core, n_points=20)
    ur_gdf = unreconstruct_geodataframe(volcano_points, reconstruction_model, reconstructed_polygons, reconstruction_time)
    ur_gdf.plot(ax=ax, color='darkred', markersize=2)
    arc_volcanism.append(ur_gdf)

    #break

plt.axis([-180,180,-90,90])
plt.show()


In [ ]:
evaporite_dataframe = gpd.GeoDataFrame(gpd.pd.concat(evaporites).reset_index())
glacial_deposits_dataframe = gpd.GeoDataFrame(gpd.pd.concat(glacial_deposits).reset_index())
arc_volcanism_dataframe = gpd.GeoDataFrame(gpd.pd.concat(arc_volcanism).reset_index())



In [ ]:
glacial_deposits_dataframe.sample(200).plot()
evaporite_dataframe.sample(200).plot()
arc_volcanism_dataframe.sample(200).plot()
